In [ ]:
import pandas as pd
from prophet import Prophet
from google.colab import files
import xlrd
import io

uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
df = pd.read_excel('/content/Recharge Data.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

m = Prophet()
m.add_seasonality(name='weekly', period=7, fourier_order=5)
m.add_seasonality(name='monthly', period=30, fourier_order=8)
m.add_seasonality(name='quarterly', period=90, fourier_order=8)
m.add_country_holidays(country_name='PAK')

m.fit(df)

df_future = m.make_future_dataframe(periods=365)
forecast = m.predict(df_future)


INFO:prophet:Found custom seasonality named 'weekly', disabling built-in 'weekly' seasonality.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_n1zgp1/emu42thi.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_n1zgp1/rdenorlr.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48897', 'data', 'file=/tmp/tmpj_n1zgp1/emu42thi.json', 'init=/tmp/tmpj_n1zgp1/rdenorlr.json', 'output', 'file=/tmp/tmpj_n1zgp1/prophet_modelsdazhpi_/prophet_model-20250327104556.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:45:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:45:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(m, initial = '365 days', period = '30 days', horizon = '30 days')

df_p = performance_metrics(df_cv)
df_p.to_excel('df_p.xlsx')
df_cv.to_excel('df_cv.xlsx')

INFO:prophet:Making 14 forecasts with cutoffs between 2024-01-16 00:00:00 and 2025-02-09 00:00:00


  0%|          | 0/14 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_n1zgp1/wtf74_0_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_n1zgp1/d4dddqjw.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=60887', 'data', 'file=/tmp/tmpj_n1zgp1/wtf74_0_.json', 'init=/tmp/tmpj_n1zgp1/d4dddqjw.json', 'output', 'file=/tmp/tmpj_n1zgp1/prophet_model59mrqe42/prophet_model-20250327104601.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:46:01 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:46:01 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_n1zgp1/pll9bkal.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_n1zgp1/1orrok0c.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [ ]:
from google.colab import drive

forecast.to_excel('/content/drive/MyDrive/forecast.xlsx', index=False)

In [10]:
!git clone https://github.com/Sannan27/glowing-adventure.git

%cd glowing-adventure

!git config user.name "abc"
!git config user.email "abc"

!git add .
!git commit -m"Recharge Forecast v 1.0"
!git push

Cloning into 'glowing-adventure'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
/content/glowing-adventure
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [9]:
%cd /content
!rm -rf glowing-adventure

/content
